In [1]:
import pyspark
from pyspark.sql import SparkSession, types
from pyspark.sql.functions import col, when
import pyspark.sql.types as T
import pyspark.sql.functions as F 
import pandas

In [2]:
spark = SparkSession.builder.master('local[*]').appName('risk').getOrCreate()

In [3]:
df_data = spark.read.csv('./Data/TD_Expert_Selection_D4_I5_R0.csv',header=True)
df_data.show(1,truncate=False)

+-----------+---+---+---+---+---+---+---+------+
|d1         |d2 |d3 |d4 |d5 |d6 |d7 |d8 |action|
+-----------+---+---+---+---+---+---+---+------+
|5589.792219|0  |0  |0  |0  |0  |0  |0  |H0    |
+-----------+---+---+---+---+---+---+---+------+
only showing top 1 row



In [4]:
df_data = df_data.withColumn('action',F.substring(F.col('action'),-1,1)).withColumnRenamed('action', 'heuristic')
df_data.show(2,truncate=False)
df_data.toPandas().to_csv('./Data/dataset.csv',index=None)

+-----------+-----------+---+---+---+---+---+---+---------+
|d1         |d2         |d3 |d4 |d5 |d6 |d7 |d8 |heuristic|
+-----------+-----------+---+---+---+---+---+---+---------+
|5589.792219|0          |0  |0  |0  |0  |0  |0  |0        |
|0          |5589.792219|0  |0  |0  |0  |0  |0  |1        |
+-----------+-----------+---+---+---+---+---+---+---------+
only showing top 2 rows

